# Azure ML Workspace

## Importing libraries

In [1]:
from azureml.core import Workspace, Environment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 3.1.1 (c:\users\jack\anaconda3\lib\site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).


## Initialise workspace
Initialise a workspace using the values from config.json
If this file is not found, an exception occurs
When an exception is detected, a workspace using the defined details is created

In [2]:
try:
    ## Try to load workspace
    wspace = Workspace.from_config()
except Exception as ex:
    ## Create new workspace if existing is not found
    wspace = Workspace.create(name = "ml-music-genre-classification",
                                  subscription_id = "11636c4c-f408-4202-980a-b738e97f4d7f",
                                  resource_group = "final-year-project",
                                  create_resource_group = True,
                                  location = "uksouth")

    ## Write workspace config to a local file
    wspace.write_config()

print("Workspace successfully loaded")

get_workspace error using subscription_id=11636c4c-f408-4202-980a-b738e97f4d7f, resource_group_name=final-year-project, workspace_name=ml-music-genre-classification


Deploying AppInsights with name mlmusicginsights5831c13c.
Deployed AppInsights with name mlmusicginsights5831c13c. Took 3.42 seconds.
Deploying KeyVault with name mlmusicgkeyvault819ef2b6.
Deploying StorageAccount with name mlmusicgstorage37faa0b13.
Deployed KeyVault with name mlmusicgkeyvault819ef2b6. Took 17.98 seconds.
Deploying Workspace with name ml-music-genre-classification.
Deployed StorageAccount with name mlmusicgstorage37faa0b13. Took 20.79 seconds.
Deployed Workspace with name ml-music-genre-classification. Took 19.12 seconds.
Workspace successfully loaded


## Create an environment
Create an environment for the model to run on the Azure servers

In [3]:
##  Load conda dependencies required for this environment.
##  Register environment for the workspace
env = Environment("env")

conda_dep = CondaDependencies.create(pip_packages = ["pandas == 1.1.3",
                                                    "azureml-core",
                                                    'azureml-defaults >= 1.0.45'],
                                    conda_packages = ["scikit-learn == 0.23.2"])

env.python.conda_dependencies = conda_dep

env.register(workspace = wspace)

print("Environment Registered")

## Get the environment created from the workspace and create a file to verify it has been defined
my_env = Environment.get(workspace = wspace, name = "env")
my_env.save_to_directory("./environ", overwrite = True)

print("Environment Saved")

inference_config = InferenceConfig(entry_script = "score.py",
                                   environment = my_env)



Environment Registered
Environment Saved


## Register the model
Register the saved model created by "knn-model-tuning" for use on Azure ML

In [4]:
model = Model.register(workspace = wspace,
                       model_name = "knn",
                       model_path = "models/knn.pkl",
                       model_framework = "ScikitLearn",
                       description = "Tuned KNN for Genre Classification")

Registering model knn


## Set up Azure Container Instance
Define ACI Webservice

In [5]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1)

## Deploy the model
Deploy the model onto the defined ACI container using the set environment, inference config and workspace

In [6]:
service = Model.deploy(workspace = wspace,
                       name = "genre-classification",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = aci_config,
                       overwrite = True)

service.wait_for_deployment(show_output = True)
print(service.get_logs())

WebserviceException: WebserviceException:
	Message: There is no model with id knn:1 in ml-music-genre-classification
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is no model with id knn:1 in ml-music-genre-classification"
    }
}